In [ ]:
import pathlib
import glob
import os
import collections
import json
from omegaconf import OmegaConf

import pickle
import sys
import glob
from pathlib import Path, PurePath
path = Path.cwd()
parent_path = path.parents[1]
sys.path.append(str(PurePath(parent_path, 'neuroformer')))
sys.path.append('neuroformer')
sys.path.append('.')
sys.path.append('../')

import pandas as pd
import numpy as np
from einops import rearrange

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data.dataloader import DataLoader

import math
from torch.utils.data import Dataset

from neuroformer.model_neuroformer import GPT, GPTConfig, neuralGPTConfig
from neuroformer.trainer import Trainer, TrainerConfig
from neuroformer.utils import set_seed, update_object, check_common_attrs
from neuroformer.visualize import set_plot_params, set_plot_white
from neuroformer.SpikeVidUtils import round_n, set_intervals
set_plot_params()
set_plot_white()

from scipy import io as scipyio
from scipy.special import softmax
import skimage
import skvideo.io
from scipy.ndimage import gaussian_filter, uniform_filter

parent_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/"
import argparse


In [ ]:
def align_trials(df_glm, t_data, key='raw_interval'):
    for i, trial in enumerate(t_data['Trial'].unique()):
        start_interval = t_data[t_data['Trial'] == trial][key].min()
        end_interval = t_data[t_data['Trial'] == trial][key].max()
        # print(start_interval, end_interval, trial)
        df_glm.loc[(df_glm[key] >= start_interval) & (df_glm[key] <= end_interval), 'Trial'] = trial
    return df_glm

In [ ]:
from neuroformer.SpikeVidUtils import create_full_trial
from neuroformer.SpikeVidUtils import make_intervals
from neuroformer.analysis import get_rates_trial, calc_corr_psth, get_accuracy, compute_scores

DATASET = "V1_AL"
T_BIN = 1

In [ ]:
data_path = "./data/Combo3_V1AL/Combo3_V1AL_response.csv"
df = pd.read_csv(data_path)
df['Interval'] = make_intervals(df, T_BIN)

pred_path = "./data/Combo3_V1AL/glm_comparison/pred.csv"
df_pred = pd.read_csv(pred_path)

In [ ]:
glm_path = "./data/Combo3_V1AL/combo3neuronGLM.mat"
mat = scipyio.loadmat(glm_path)
df_glm = pd.DataFrame(mat['glmneuron'], columns=['Interval', 'ID', 'Trial']).reset_index(drop=True)
df_glm['Time'] = df_glm['Interval']
df_glm = df_glm.sort_values(by=['Trial', 'Interval']).reset_index(drop=True)
df_glm = align_trials(df_glm, df_pred, key='Interval')
df_glm['Interval'] = make_intervals(df_glm, T_BIN)

In [ ]:
# trial ranges trial 0-20:(0 - 32), trial 20-40 (32 - 64), trial 40-60:(64 - 96)

df['Interval'][df['Trial'] > 20] = df['Interval'][df['Trial'] > 20] + 32
df_pred['Interval'][df_pred['Trial'] > 20] = df_pred['Interval'][df_pred['Trial'] > 20] + 32
df_glm['Interval'][df_glm['Trial'] > 20] = df_glm['Interval'][df_glm['Trial'] > 20] + 32

df['Interval'][df['Trial'] > 40] = df['Interval'][df['Trial'] > 40] + 32
df_pred['Interval'][df_pred['Trial'] > 40] = df_pred['Interval'][df_pred['Trial'] > 40] + 32
df_glm['Interval'][df_glm['Trial'] > 40] = df_glm['Interval'][df_glm['Trial'] > 40] + 32



In [ ]:
df_glm = df_glm.sort_values(by=['Trial', 'Interval']).reset_index(drop=True)    
print(df_glm['Trial'].unique())

In [ ]:
trials = df_pred['Trial'].unique()
n_1 = trials + 1
n_2 = trials + 2

In [ ]:
from neuroformer.SpikeVidUtils import create_full_trial

df_pred = create_full_trial(df_pred, trials)
df_1 = create_full_trial(df, trials)
df_2 = create_full_trial(df, n_1)
df_3 = create_full_trial(df, n_2)
df_glm = create_full_trial(df_glm, trials)

df_pred['Interval'] = make_intervals(df_pred, T_BIN)
df_1['Interval'] = make_intervals(df_1, T_BIN)
df_2['Interval'] = make_intervals(df_2, T_BIN)
df_3['Interval'] = make_intervals(df_3, T_BIN)

# trim everything to length L
# L = 100
# df_pred = df_pred.iloc[:L]
# df_1 = df_1.iloc[:L]
# df_2 = df_2.iloc[:L]
# df_3 = df_3.iloc[:L]

In [ ]:
from neuroformer.analysis import compute_scores, compute_scores_scikit
scores = compute_scores(df_1, df_pred)
scores_scikit = compute_scores_scikit(df_1, df_pred)

scores_glm = compute_scores(df_1, df_glm)
scores_scikit_glm = compute_scores_scikit(df_1, df_glm)

print("Scores")
print("Neuroformer")
print(scores)
print("GLM")
print(scores_glm)

print("Scores Scikit")
print("Neuroformer")
print(scores_scikit)
print("GLM")
print(scores_scikit_glm)

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming your dataframes are named 'pred' and 'true'

# Define the number of bins
true = df_1
pred = df_pred

# Bin the 'Interval' values into classes
pred['Interval_pred'] = pd.Categorical(pred['Interval']).codes
true['Interval_true'] = pd.Categorical(true['Interval']).codes

# Merge your dataframes on 'ID' and 'Trial' to align predictions with ground truth
merged_df = pd.merge(pred, true, on=['ID', 'Trial'])

# Then group by 'ID' and 'Trial'
grouped = merged_df.groupby(['ID', 'Trial'])

# Initialize lists to store results
ids = []
trials = []
precisions = []
recalls = []
f1s = []

# For each group, calculate precision, recall, and F1 score
for name, group in grouped:
    id, trial = name
    precision = precision_score(group['Interval_true'], group['Interval_pred'], average='micro')
    recall = recall_score(group['Interval_true'], group['Interval_pred'], average='micro')
    f1 = f1_score(group['Interval_true'], group['Interval_pred'], average='micro')
    
    ids.append(id)
    trials.append(trial)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Create a results DataFrame
results = pd.DataFrame({
    'ID': ids,
    'Trial': trials,
    'Precision': precisions,
    'Recall': recalls,
    'F1': f1s
})

mean_results = results.mean()

print(results.mean())


In [ ]:
print(mean_results)

In [ ]:
df_pred_full = df_pred

window = 0.5
window_prev = 0.5
window_pred = 0.5
labels = np.array([round(window_pred + window_pred*n, 2) for n in range(0, int(max(df_pred_full['Interval']) / window_pred))])

df_1 = set_intervals(df_1, window, window_prev, window_pred)
df_2 = set_intervals(df_2, window, window_prev, window_pred)
df_3 = set_intervals(df_3, window, window_prev, window_pred)

rates_pred = get_rates_trial(df_pred_full, labels)
rates_1 = get_rates_trial(df_1, labels)
rates_2 = get_rates_trial(df_2, labels)
rates_3 = get_rates_trial(df_3, labels)
rates_glm = get_rates_trial(df_glm, labels)

neurons = df['ID'].unique()
top_corr_pred = calc_corr_psth(rates_pred, rates_1, neurons=neurons)
top_corr_real = calc_corr_psth(rates_1, rates_2, neurons=neurons)
top_corr_real_2 = calc_corr_psth(rates_1, rates_3, neurons=neurons)
top_corr_glm = calc_corr_psth(rates_glm, rates_1, neurons=neurons)

# turn index into ID column
top_corr_pred = top_corr_pred.reset_index()
top_corr_real = top_corr_real.reset_index()
top_corr_glm = top_corr_glm.reset_index()

# rename columns
top_corr_pred = top_corr_pred.rename(columns={'index': 'ID', 0: 'Correlation'})
top_corr_real = top_corr_real.rename(columns={'index': 'ID', 0: 'Correlation'})
top_corr_glm = top_corr_glm.rename(columns={'index': 'ID', 0: 'Correlation'})

In [ ]:
top_corr_pred

In [ ]:
# # turn index into ID column
# top_corr_pred = top_corr_pred.reset_index()
# top_corr_real = top_corr_real.reset_index()
# top_corr_glm = top_corr_glm.reset_index()

# # rename columns
# top_corr_pred = top_corr_pred.rename(columns={'index': 'ID', 0: 'Correlation'})
# top_corr_real = top_corr_real.rename(columns={'index': 'ID', 0: 'Correlation'})
# top_corr_glm = top_corr_glm.rename(columns={'index': 'ID', 0: 'Correlation'})

In [ ]:
# find duplicate values in IDF

In [ ]:
merged_df

In [ ]:
# Merge dataframes on 'ID'
merged_df = pd.merge(top_corr_pred, top_corr_glm, on=['ID'])

# Create a scatter plot
plt.scatter(merged_df['pearson_r_y'], merged_df['pearson_r_x'], alpha=0.7)

# Add a reference line (y=x) to show where the predicted correlation equals the true correlation
plt.plot(
    [merged_df['pearson_r_y'].min(), merged_df['pearson_r_y'].max()],
    [merged_df['pearson_r_y'].min(), merged_df['pearson_r_y'].max()],
    color='red', linestyle='--'
)

# Add axis labels and a title
plt.xlabel('True Correlation')
plt.ylabel('Predicted Correlation')
plt.title('Predicted vs. True Correlation')

# Show the plot
plt.show()

In [ ]:
top_corr_pred

In [ ]:
top_ids = top_corr_pred.sort_values(by=['pearson_r'], ascending=False).iloc[:10]
print(top_ids)

In [ ]:
# plot spikes for top 10 neurons
trial_duration = 96
n_intervals = int(trial_duration / window_pred)
intervals = np.array([round(window_pred + window_pred*n, 2) for n in range(0, n_intervals)])

print(intervals)

In [ ]:
glm_path = "./data/Combo3_V1AL/combo3neuronGLM.mat"
mat = scipyio.loadmat(glm_path)
df_glm = pd.DataFrame(mat['glmneuron'], columns=['Interval', 'ID', 'Trial']).reset_index(drop=True)
df_glm['Time'] = df_glm['Interval']
df_glm = df_glm.sort_values(by=['Trial', 'Interval']).reset_index(drop=True)
df_glm = align_trials(df_glm, df_pred, key='Interval')
df_glm['Interval'] = make_intervals(df_glm, T_BIN)

In [ ]:
fig, axes = plt.subplots(20, 1, figsize=(10, 30), sharex=True)  # Create subplots with shared x-axis

for i in range(0, 10):
    neuron_id = top_ids.iloc[i]['ID']
    neuron_pred = df_pred[df_pred['ID'] == neuron_id]
    trials = neuron_pred['Trial'].unique()
    neuron_true = df[(df['ID'] == neuron_id) & (df['Trial'].isin(trials))]
    neuron_pred_times = neuron_pred['Time']
    neuron_true_times = neuron_true['Time']
    neuron_pred = neuron_pred.sort_values(by=['Time'])
    neuron_true = neuron_true.sort_values(by=['Time'])
    
    # Predicted plot on top
    axes[2*i].scatter(neuron_pred['Interval'], neuron_pred['Trial'], color='k', marker='|', label='Predicted')
    axes[2*i].set_xlim([0, 96])  # Setting x limits
    axes[2*i].set_ylabel('Trial', fontsize=13)
    
    # True plot at bottom
    axes[2*i+1].scatter(neuron_true['Interval'], neuron_true['Trial'], color='r', marker='|', label='True')
    axes[2*i+1].set_xlim([0, 96])  # Setting x limits
    axes[2*i+1].set_ylabel('Trial', fontsize=13)

plt.xlabel('Interval', fontsize=15)
plt.suptitle('Neuron Spike Rasters', fontsize=20, y=0.91)  # Main title for all subplots
plt.show()

In [ ]:
df_glm

In [ ]:
fig, axes = plt.subplots(30, 1, figsize=(10, 60), sharex=True)  # Create subplots with shared x-axis

for i in range(0, 10):
    neuron_id = top_ids.iloc[i]['ID']
    neuron_pred = df_pred[df_pred['ID'] == neuron_id]
    neuron_glm_pred = df_glm[df_glm['ID'] == neuron_id]
    trials = neuron_pred['Trial'].unique()
    neuron_true = df[(df['ID'] == neuron_id) & (df['Trial'].isin(trials))]
    neuron_pred_times = neuron_pred['Time']
    neuron_glm_pred_times = neuron_glm_pred['Time']
    neuron_true_times = neuron_true['Time']
    neuron_pred = neuron_pred.sort_values(by=['Time'])
    neuron_glm_pred = neuron_glm_pred.sort_values(by=['Time'])
    neuron_true = neuron_true.sort_values(by=['Time'])

    # Predicted plot on top
    axes[3*i].scatter(neuron_pred['Interval'], neuron_pred['Trial'], color='k', marker='|', label='Predicted')
    axes[3*i].set_xlim([0, 100])  # Setting x limits
    axes[3*i].set_ylabel('Trial', fontsize=13)
    axes[3*i].set_yticks([])  # Disable ytick labels
    axes[3*i].set_yticks(np.linspace(0, neuron_pred['Trial'].max(), 4))  # Keep 4 ticks on the y-axis

    # True plot at bottom
    axes[3*i+2].scatter(neuron_glm_pred['Interval'], neuron_glm_pred['Trial'], color='b', marker='|', label='GLM Predicted')
    axes[3*i+2].set_xlim([0, 100])  # Setting x limits
    axes[3*i+2].set_ylabel('Trial', fontsize=13)
    axes[3*i+2].set_yticks([])  # Disable ytick labels
    axes[3*i+2].set_yticks(np.linspace(0, neuron_glm_pred['Trial'].max(), 4))  # Keep 4 ticks on the y-axis

    # GLM Predicted plot in the middle
    axes[3*i+1].scatter(neuron_true['Interval'], neuron_true['Trial'], color='r', marker='|', label='True')
    axes[3*i+1].set_xlim([0, 100])  # Setting x limits
    axes[3*i+1].set_ylabel('Trial', fontsize=13)
    axes[3*i+1].set_yticks([])  # Disable ytick labels
    axes[3*i+1].set_yticks(np.linspace(0, neuron_true['Trial'].max(), 4))  # Keep 4 ticks on the y-axis

    # Add overarching title
    axes[3*i].set_title('Neuron ID: ' + str(neuron_id), fontsize=15, y=1.05)
    plt.subplots_adjust(hspace=0.5)


# disable all ytick labels
for ax in axes:
    ax.set_yticklabels([])
    ax.set_yticks([])
    
plt.xlabel('Interval', fontsize=15)
plt.suptitle('Neuron Spike Rasters', fontsize=20, y=0.91)  # Main title for all subplots

plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
for i in range(0, 10):
    neuron_id = top_ids.iloc[i]['ID']
    neuron_pred = df_pred[df_pred['ID'] == neuron_id]
    trials = neuron_pred['Trial'].unique()
    neuron_true = df[(df['ID'] == neuron_id) & (df['Trial'].isin(trials))]
    neuron_pred_times = neuron_pred['Time']
    neuron_true_times = neuron_true['Time']
    neuron_pred = neuron_pred.sort_values(by=['Time'])
    neuron_true = neuron_true.sort_values(by=['Time'])
    plt.subplot(5, 2, i+1)  # Displaying 5 rows, 2 columns of plots.
    plt.scatter(neuron_pred['Interval'], neuron_pred['Trial'], color='k', marker='|', label='Predicted')
    plt.scatter(neuron_true['Interval'], neuron_true['Trial'], color='r', marker='|', label='True')
    plt.title('Neuron ID: ' + str(neuron_id), fontsize=15)
    plt.xlabel('Interval', fontsize=13)
    plt.ylabel('Trial', fontsize=13)
    plt.legend()
plt.tight_layout()  # To ensure the subplots do not overlap
plt.suptitle('Neuron Spike Rasters', fontsize=20, y=1.02)  # Main title for all subplots
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))

for i in range(0, 10):
    neuron_id = top_ids.iloc[i]['ID']
    neuron = df_pred[df_pred['ID'] == neuron_id]
    neuron_times = neuron['Time']
    neuron = neuron.sort_values(by=['Time'])
    plt.subplot(5, 2, i+1)  # Displaying 5 rows, 2 columns of plots.
    
    # Define the number of bins to suit your data
    plt.hist2d(neuron['Interval'], neuron['Trial'], bins=[100, 100], cmap='binary')
    
    plt.title('Neuron ID: ' + str(neuron_id), fontsize=15)
    plt.xlabel('Interval', fontsize=13)
    plt.ylabel('Trial', fontsize=13)
    plt.colorbar(label='Number of Spikes')  # Shows the color scale
    
plt.tight_layout()  # To ensure the subplots do not overlap
plt.suptitle('Neuron Spike Histograms', fontsize=20, y=1.02)  # Main title for all subplots
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))

for i in range(0, 10):
    neuron_id = top_ids.iloc[i]['ID']
    neuron = df_pred[df_pred['ID'] == neuron_id]
    neuron_times = neuron['Time']
    neuron = neuron.sort_values(by=['Time'])
    plt.subplot(5, 2, i+1)  # Displaying 5 rows, 2 columns of plots.
    
    # Define the number of bins to suit your data
    plt.hist(neuron['Interval'], bins=100, color='black')
    
    plt.title('Neuron ID: ' + str(neuron_id), fontsize=15)
    plt.xlabel('Interval', fontsize=13)
    plt.ylabel('Number of Spikes', fontsize=13)
    
plt.tight_layout()  # To ensure the subplots do not overlap
plt.suptitle('Neuron Spike Histograms across Intervals', fontsize=20, y=1.02)  # Main title for all subplots
plt.show()